In [1]:
import html
from lxml import etree
import lxml.html
import re
from slugify import slugify

In [2]:
FILE = 'publications-2021-website.html'
DOWNLOAD_PREFIX = 'https://op.europa.eu'

In [3]:
with open(FILE, encoding="utf-8") as inf:
    doc = lxml.html.fromstring(inf.read())

In [4]:
doc[0:100]

[<Element head at 0x7f9294f992f0>, <Element body at 0x7f9294f991d0>]

In [5]:
results = doc.cssselect("ul.search-results-items > li.list-item")

In [6]:
len(results)

48

In [7]:
publications = []
for li in results:
    name = li.xpath('./h2//a/span[@itemprop="name"]/text()')
    authors = list(filter(lambda x: len(x) > 0, li.xpath(".//span[@itemprop='author']/a/text()")))
    slugified_authors = list(map(lambda x: slugify(x), authors))
    date = li.xpath(".//li/time[@itemprop='datePublished']/@datetime")
    download_elems = li.xpath('.//a[@class="download"]/@data-uri')
    
    if len(download_elems):
        download = DOWNLOAD_PREFIX + li.xpath('.//a[@class="download"]/@data-uri')[0]
    else:
        download = 'NO_DOWNLOAD'
    
    p = [name, authors, date, download, slugified_authors]
    publications.append(p)

publications[0]

[['REGULATION OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL ESTABLISHING THE CUSTOMS PROGRAMME FOR COOPERATION IN THE FIELD OF CUSTOMS AND REPEALING REGULATION (EU) No 1294/2013'],
 ['Council of the European Union', 'European Parliament'],
 ['2021-03-11'],
 'https://op.europa.eu/o/opportal-service/download-handler?identifier=20ef3c20-834a-11eb-af5d-01aa75ed71a1&format=PDF&language=en&productionSystem=cellar&documentIdentifier=20ef3c20-834a-11eb-af5d-01aa75ed71a1.0006',
 ['council-of-the-european-union', 'european-parliament']]

In [ ]:
import json
publications_json = []

In [10]:
publications_json = []
for p in publications:
    
    if not len(p[0]):
        print(p)
    
    p_json = {
        "name": p[0][0] if len(p[0]) else "Why is terminology your passion? The fifth collection of interviews with prominent terminologists",
        #"name": p[0][0],
        "authors": p[1],
        "date": p[2][0],
        "download": p[3]
    }
    
    publications_json.append(p_json)

publications_json

[[], ['European Parliament'], ['2021-02-12'], 'https://op.europa.eu/o/opportal-service/download-handler?identifier=f9ed916d-7007-11eb-9ac9-01aa75ed71a1&format=PDF&language=en&productionSystem=cellar&documentIdentifier=f9ed916d-7007-11eb-9ac9-01aa75ed71a1.0001', ['european-parliament']]


[{'name': 'REGULATION OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL ESTABLISHING THE CUSTOMS PROGRAMME FOR COOPERATION IN THE FIELD OF CUSTOMS AND REPEALING REGULATION (EU) No 1294/2013',
  'authors': ['Council of the European Union', 'European Parliament'],
  'date': '2021-03-11',
  'download': 'https://op.europa.eu/o/opportal-service/download-handler?identifier=20ef3c20-834a-11eb-af5d-01aa75ed71a1&format=PDF&language=en&productionSystem=cellar&documentIdentifier=20ef3c20-834a-11eb-af5d-01aa75ed71a1.0006'},
 {'name': 'Corrigendum to Recruitment notice PE/255/S – English-language clear language professional (AD 7) (female or male) (Official Journal of the European Union C 74 A of 3\xa0March 2021) 2021/C 76 I/24',
  'authors': ['European Parliament'],
  'date': '2021-03-05',
  'download': 'https://op.europa.eu/o/opportal-service/download-handler?identifier=7cde00a7-7d58-11eb-9ac9-01aa75ed71a1&format=PDF&language=en&productionSystem=cellar&documentIdentifier=7cde00a7-7d58-11eb-9ac9-01aa7

In [12]:
import json
with open("publications.json", "w", encoding="utf-8") as outf:
    json.dump(publications_json, outf)